Introduction

This notebook trains and saves a machine learning model using an LSTM architecture for predicting stock prices. The focus is on individual tickers.

Key Steps:

1.	Load processed data from SQLite.

2.	Filter data for a specific ticker (default: XOM).

3.	Preprocess and normalize features.

4.	Build and train an LSTM model.

5.	Save the trained LSTM model and scaler for use in evaluation and predictions.

Import Libraries

•	pandas and sqlite3: For data manipulation and interaction with SQLite.

•	keras and tensorflow: For building and training the LSTM model.

•	scikit-learn: For preprocessing and model evaluation.

•	joblib: For saving trained models and scalers.

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import sqlite3
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


Load Processed Data

•	Load the preprocessed stock data from the SQLite database (stocks_data.db).

•	Ensure the dataset is ready for filtering and training.

In [2]:

# Path to SQLite database
db_path = 'database/stocks_data.db'

# Load data from SQLite
with sqlite3.connect(db_path) as conn:
    query = "SELECT * FROM processed_stocks"
    data = pd.read_sql(query, conn)
print(f"Loaded processed data: {data.shape[0]} rows")


Loaded processed data: 67834 rows


Filter Data for Default Ticker

•	Filter data to include only rows for the default ticker (e.g., XOM).

•	Verify the number of rows available for the selected ticker.

In [3]:

# Step 1: Set default ticker
default_ticker = 'XOM'

# Step 2: Filter data for the default ticker
ticker_data = data[data['Ticker'] == default_ticker]
print(f"Loaded data for {default_ticker}: {ticker_data.shape[0]} rows")


Loaded data for XOM: 11791 rows


Define Features and Target

•	Define independent variables (features) for predictions: ['7-day MA', '14-day MA', 'Volatility', 'Lag_1', 'Lag_2'].

•	Define the dependent variable (target): Adj Close.

•	Prepare the dataset for model training.

In [4]:

# Define features and target
features = ['7-day MA', '14-day MA', 'Volatility', 'Lag_1', 'Lag_2']
target = 'Adj Close'

X = ticker_data[features]
y = ticker_data[target]


Normalize Features

•	Use MinMaxScaler to scale the features between 0 and 1.

•	Fit the scaler on the training data and apply the transformation to both training and testing datasets.

In [5]:

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


Split Data into Training and Testing Sets

•	Divide the dataset into:

•	Training Set: 80% of the data, used for model training.

•	Testing Set: 20% of the data, used for model evaluation.

•	Ensures fair evaluation of the model’s predictive performance.

In [6]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


Reshape Input for LSTM

•	Reshape the input data to match the expected format for LSTM models: (samples, timesteps, features).

•	In this case, timesteps=1 as each prediction is based on one timestep.

In [7]:

# Reshape input for LSTM (samples, timesteps, features)
X_train_scaled = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_scaled = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


Build and Compile the LSTM Model

•	Model Architecture:

•	First LSTM layer with 64 neurons, relu activation, and return_sequences=True to allow stacking.

•	Dropout layer to reduce overfitting.

•	Second LSTM layer with 32 neurons and another Dropout layer.

•	Fully connected Dense output layer with 1 neuron for regression.

•	Compilation:

•	Optimizer: adam

•	Loss function: mean_squared_error

In [8]:

# Build the optimized LSTM model
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(1, X_train_scaled.shape[2])))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))  # Fully connected output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


c:\Users\ELCACAZ\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Train the LSTM Model

•	Use EarlyStopping to stop training if validation loss does not improve after 10 epochs.

•	Train the model for up to 100 epochs with a batch size of 128.

In [9]:
# Callbacks for better training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=50,  # Start with 100 epochs
    batch_size=32,  # increased batch size
    callbacks=[early_stopping]
)


Epoch 1/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1972.2327 - val_loss: 7.5620
Epoch 2/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.9595 - val_loss: 4.7160
Epoch 3/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.0516 - val_loss: 4.1720
Epoch 4/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21.9134 - val_loss: 3.4493
Epoch 5/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19.8974 - val_loss: 2.3727
Epoch 6/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24.4500 - val_loss: 4.8111
Epoch 7/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.6548 - val_loss: 2.6236
Epoch 8/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.0319 - val_loss: 2.0844
Epoch 9/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 19.5466 - val_loss: 1.9469
Epoch 10/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.3499 - val_loss: 3.0888
Epoch 11/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.3894 - val_loss: 2.4215
Epoch 12/50
295/295 ━━━━━━━━━━━━━━━━━━━

Evaluate the Model

•	Use the test data to evaluate the model’s performance:

•	Mean Squared Error (MSE): Measures average squared differences between actual and predicted values.

•	Mean Absolute Error (MAE): Measures the average magnitude of errors.

•	R-squared (R²): Proportion of variance explained by the model.

In [10]:

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")


74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Model Evaluation:
Mean Squared Error (MSE): 1.09
Mean Absolute Error (MAE): 0.73
R-squared (R²): 1.00


Save the Model and Scaler

•	Save the trained LSTM model as model_<TICKER>_lstm.h5.

•	Save the fitted scaler as scaler_<TICKER>_lstm.pkl.

In [11]:

# Save the trained model and scaler
model_filename = f'models/model_{default_ticker}_lstm.h5'
scaler_filename = f'models/scaler_{default_ticker}_lstm.pkl'

model.save(model_filename)
joblib.dump(scaler, scaler_filename)

print(f"{default_ticker} model saved as '{model_filename}'")
print(f"{default_ticker} scaler saved as '{scaler_filename}'")

XOM model saved as 'models/model_XOM_lstm.h5'
XOM scaler saved as 'models/scaler_XOM_lstm.pkl'


Next Steps

1.	Extend the Flask app to load LSTM models dynamically based on the selected ticker.

2.	Create additional visualizations for residuals and predicted values.

3.	Evaluate the model’s performance across multiple tickers to ensure robustness.